In [19]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import hvplot.pandas
from fractions import Fraction
pd.set_option('display.max_columns', None)

In [20]:
# Import dataframes
results_df = pd.read_csv(Path('./Resources/week_13_base_results.csv'))

In [21]:
results_df

,week,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner,Accuracy
0,13,Arizona Cardinals,Chicago Bears,0.895526,0.104474,Arizona Cardinals,Arizona Cardinals,1
1,13,Baltimore Ravens,Pittsburgh Steelers,0.927091,0.072909,Baltimore Ravens,Pittsburgh Steelers,0
2,13,Dallas Cowboys,New Orleans Saints,0.960130,0.039870,Dallas Cowboys,Dallas Cowboys,1
3,13,Denver Broncos,Kansas City Chiefs,0.006560,0.993440,Kansas City Chiefs,Kansas City Chiefs,1
4,13,Indianapolis Colts,Houston Texans,0.826016,0.173984,Indianapolis Colts,Indianapolis Colts,1
5,13,Jacksonville Jaguars,Los Angeles Rams,0.064065,0.935935,Los Angeles Rams,Los Angeles Rams,1
6,13,Los Angeles Chargers,Cincinnati Bengals,0.159052,0.840948,Cincinnati Bengals,Los Angeles Chargers,0
7,13,Minnesota Vikings,Detroit Lions,0.316795,0.683205,Detroit Lions,Detroit Lions,1
8,13,New England Patriots,Buffalo Bills,0.999073,0.000926,New England Patriots,New England Patriots,1
9,13,New York Giants,Miami Dolphins,0.048817,0.951183,Miami Dolphins,Miami Dolphins,1


In [22]:
#American Odds Converter
def american_odds_converter(away_team_win_pct, home_team_win_pct):
    if home_team_win_pct - away_team_win_pct > 0:
        home_team_american_odds = round(((home_team_win_pct/(1-home_team_win_pct))*100)*(-1),0)
        away_team_american_odds = round(((100/(away_team_win_pct))-100),0)
    elif home_team_win_pct - away_team_win_pct < 0:
        away_team_american_odds = round(((away_team_win_pct/(1-away_team_win_pct))*100)*(-1),0)
        home_team_american_odds = round(((100/(home_team_win_pct))-100),0)
    else:
        away_team_american_odds = 100
        home_team_american_odds = 100
    return away_team_american_odds, home_team_american_odds

#Fractional Odds Converter
def fractional_odds_converter(away_team_win_pct, home_team_win_pct):
    away_team_american_odds, home_team_american_odds = american_odds_converter(away_team_win_pct, home_team_win_pct)
    if home_team_american_odds < 0:
        away_team_fractional_odds = (Fraction(round(away_team_american_odds / 100, 0)))
        home_team_fractional_odds = (Fraction(round(((-100)/home_team_american_odds)*16)/16))
    elif home_team_american_odds > 0:
        home_team_fractional_odds = (Fraction(round(home_team_american_odds / 100, 0)))
        away_team_fractional_odds = (Fraction(round(((-100)/away_team_american_odds)*16)/16))
    else: 
        away_team_fractional_odds = Fraction(1)
        home_team_fractional_odds = Fraction(1)
    return away_team_fractional_odds, home_team_fractional_odds     

#Decimal Odds Converter
def decimal_odds_converter(away_team_win_pct, home_team_win_pct):
    away_team_american_odds, home_team_american_odds = american_odds_converter(away_team_win_pct, home_team_win_pct)
    if home_team_american_odds < 0:
        away_team_decimal_odds = round((1 + (away_team_american_odds/100)),2)
        home_team_decimal_odds = round((1-(100/home_team_american_odds)),2)
    elif home_team_american_odds > 0:
        home_team_decimal_odds = round((1 + (home_team_american_odds/100)),2)
        away_team_decimal_odds = round((1-(100/away_team_american_odds)),2)
    else: 
        away_team_decimal_odds = 1
        home_team_decimal_odds = 1
    return away_team_decimal_odds, home_team_decimal_odds      

In [23]:
away_team_american_odds, home_team_american_odds = american_odds_converter(results_df.iloc[7]['away_team_win_%'], results_df.iloc[7]['home_team_win_%'])

away_team_fractional_odds, home_team_fractional_odds = fractional_odds_converter(results_df.iloc[7]['away_team_win_%'], results_df.iloc[7]['home_team_win_%'])

away_team_decimal_odds, home_team_decimal_odds = decimal_odds_converter(results_df.iloc[7]['away_team_win_%'], results_df.iloc[7]['home_team_win_%'])

display(results_df.iloc[7]['away_team_win_%'])
display(results_df.iloc[7]['home_team_win_%'])
display(away_team_american_odds)
display(home_team_american_odds)
display(away_team_fractional_odds)
display(home_team_fractional_odds)
display(away_team_decimal_odds)
display(home_team_decimal_odds)


0.31679475

0.68320525

216.0

-216.0

Fraction(2, 1)

Fraction(7, 16)

3.16

1.46

In [24]:
projected_odds = results_df.copy()

In [25]:
probability = results_df.drop(columns = ['week','away_name' ,'home_name', 'predicted_winner', 'actual_winner', 'Accuracy'])

In [26]:
probability['American Odds (Away Team, Home Team)'] = probability.apply(lambda x: american_odds_converter(x['away_team_win_%'], x['home_team_win_%']), axis=1)

In [27]:
probability['Fractional Odds (Away Team, Home Team)'] = probability.apply(lambda x: fractional_odds_converter(x['away_team_win_%'], x['home_team_win_%']), axis=1)

In [28]:
probability['Decimal Odds (Away Team, Home Team)'] = probability.apply(lambda x: decimal_odds_converter(x['away_team_win_%'], x['home_team_win_%']), axis=1)

In [29]:
probability

,away_team_win_%,home_team_win_%,"American Odds (Away Team, Home Team)","Fractional Odds (Away Team, Home Team)","Decimal Odds (Away Team, Home Team)"
0,0.895526,0.104474,"(-857.0, 857.0)","(1/8, 9)","(1.12, 9.57)"
1,0.927091,0.072909,"(-1272.0, 1272.0)","(1/16, 13)","(1.08, 13.72)"
2,0.960130,0.039870,"(-2408.0, 2408.0)","(1/16, 24)","(1.04, 25.08)"
3,0.006560,0.993440,"(15144.0, -15144.0)","(151, 0)","(152.44, 1.01)"
4,0.826016,0.173984,"(-475.0, 475.0)","(3/16, 5)","(1.21, 5.75)"
5,0.064065,0.935935,"(1461.0, -1461.0)","(15, 1/16)","(15.61, 1.07)"
6,0.159052,0.840948,"(529.0, -529.0)","(5, 3/16)","(6.29, 1.19)"
7,0.316795,0.683205,"(216.0, -216.0)","(2, 7/16)","(3.16, 1.46)"
8,0.999073,0.000926,"(-107833.0, 107834.0)","(0, 1078)","(1.0, 1079.34)"
9,0.048817,0.951183,"(1948.0, -1948.0)","(19, 1/16)","(20.48, 1.05)"


In [30]:
#results_df = pd.concat([results_df, probability.drop(columns=['away_team_win_%', 'home_team_win_%'])
results_df = pd.concat([results_df, probability.drop(columns = ['away_team_win_%','home_team_win_%'])], axis =1)

In [31]:
results_df

,week,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner,Accuracy,"American Odds (Away Team, Home Team)","Fractional Odds (Away Team, Home Team)","Decimal Odds (Away Team, Home Team)"
0,13,Arizona Cardinals,Chicago Bears,0.895526,0.104474,Arizona Cardinals,Arizona Cardinals,1,"(-857.0, 857.0)","(1/8, 9)","(1.12, 9.57)"
1,13,Baltimore Ravens,Pittsburgh Steelers,0.927091,0.072909,Baltimore Ravens,Pittsburgh Steelers,0,"(-1272.0, 1272.0)","(1/16, 13)","(1.08, 13.72)"
2,13,Dallas Cowboys,New Orleans Saints,0.960130,0.039870,Dallas Cowboys,Dallas Cowboys,1,"(-2408.0, 2408.0)","(1/16, 24)","(1.04, 25.08)"
3,13,Denver Broncos,Kansas City Chiefs,0.006560,0.993440,Kansas City Chiefs,Kansas City Chiefs,1,"(15144.0, -15144.0)","(151, 0)","(152.44, 1.01)"
4,13,Indianapolis Colts,Houston Texans,0.826016,0.173984,Indianapolis Colts,Indianapolis Colts,1,"(-475.0, 475.0)","(3/16, 5)","(1.21, 5.75)"
5,13,Jacksonville Jaguars,Los Angeles Rams,0.064065,0.935935,Los Angeles Rams,Los Angeles Rams,1,"(1461.0, -1461.0)","(15, 1/16)","(15.61, 1.07)"
6,13,Los Angeles Chargers,Cincinnati Bengals,0.159052,0.840948,Cincinnati Bengals,Los Angeles Chargers,0,"(529.0, -529.0)","(5, 3/16)","(6.29, 1.19)"
7,13,Minnesota Vikings,Detroit Lions,0.316795,0.683205,Detroit Lions,Detroit Lions,1,"(216.0, -216.0)","(2, 7/16)","(3.16, 1.46)"
8,13,New England Patriots,Buffalo Bills,0.999073,0.000926,New England Patriots,New England Patriots,1,"(-107833.0, 107834.0)","(0, 1078)","(1.0, 1079.34)"
9,13,New York Giants,Miami Dolphins,0.048817,0.951183,Miami Dolphins,Miami Dolphins,1,"(1948.0, -1948.0)","(19, 1/16)","(20.48, 1.05)"


In [32]:
# Export actual_results_df to csv based on the current_week predicted.
file_path = str(f'Resources/week_{results_df.iloc[0][0]}_odds.csv')
results_df.to_csv(Path(file_path))